<a href="https://colab.research.google.com/github/eyaler/stylegan/blob/master/smiley.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Create video from camera snapshot (click image)
%tensorflow_version 1.x

from IPython.display import HTML
from google.colab.output import eval_js
from base64 import b64decode
from google.colab import files
import os
import shutil

VIDEO_HTML = """
<video autoplay
 width=800 height=600></video>
<script>
var video = document.querySelector('video')
navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)
  
var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""
def take_photo(filename='photo.jpg', quality=1):
  display(HTML(VIDEO_HTML % quality))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return len(binary)
%cd /content
shutil.rmtree('upload', ignore_errors=True)
os.makedirs('upload', exist_ok=True)
%cd upload
take_photo()

%cd /content
!git clone https://github.com/eyaler/stylegan
%cd stylegan

!python align_images.py
assert os.listdir('face'), 'did not find any faces'
!python encode_images.py
!python smile.py

from moviepy.editor import *
%cd /content
output_folder = 'output'
file = sorted([os.path.join(output_folder, file) for file in os.listdir(output_folder) if os.path.splitext(file)[1]=='.mp4'], key=os.path.getmtime)[-1]
print('playing: ' + file)
clip = VideoFileClip(file)
clip.ipython_display(height=512, autoplay=1, loop=1, maxduration=333)

In [0]:
#@title Download videos
print() #fix issue https://github.com/googlecolab/colabtools/issues/468
from google.colab import files
%cd /content
output_folder = 'output'
output_files = sorted([os.path.join(output_folder, file) for file in os.listdir(output_folder) if os.path.splitext(file)[1]=='.mp4'], key=os.path.getmtime, reverse=True)
for file in output_files:
    print('downloading: '+file)
    files.download(file)